<a href="https://colab.research.google.com/github/Devansh-S/Multiclass_text_classification/blob/master/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.preprocessing import LabelEncoder
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Input,LSTM,Embedding,Dropout,Activation
from keras import initializers, regularizers, constraints , optimizers, layers
from nltk.corpus import stopwords 
import nltk
import string
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('stopwords')
nltk.download('punkt')

In [0]:
df = pd.read_csv('train.csv')
df1 = pd.read_csv('test.csv')

In [0]:
def preprocess(df):
    text = df['review_description'].copy()
    for i in range(0,len(df)):
        if i%10000 == 0:
            print('Processed records : ',i)
        text[i] = (str(df.iloc[i]['country']).replace(" ", "") + ' ' + 
                str(df.iloc[i]['winery']).replace(" ", "") + ' ' + 
                str(df.iloc[i]['province']).replace(" ", "") + ' ' + 
                str(df.iloc[i]['region_1']).replace(" ", "") + ' ' + 
                str(df.iloc[i]['region_2']).replace(" ", "") + ' ' + 
                str(df.iloc[i]['designation']).replace(" ", "") + ' ' + 
                str(df.iloc[i]['review_description'])) 
    return text

In [0]:
def Token_filter(text1):
  punctuation = [w for w in string.punctuation]
  stop_words = list(set(stopwords.words('english')))
  stops = stop_words + punctuation
  porter = PorterStemmer()
  filtered = text1

  for i in range(len(text1)):
    if i%10000 == 0:
      print('Processed records : ',i)
    tokenized_text = word_tokenize(text1[i])
    tokens = [w.lower() for w in tokenized_text]
    final = [w for w in tokens if not w in stops] 
    filtered[i] = final
  return filtered

In [0]:
def detokenize(text):
  for i in range(len(text)):
    text[i] = TreebankWordDetokenizer().detokenize(text[i])
  return text

In [0]:
def prepare_labels(df):
  labels = df['variety']
  le = LabelEncoder()
  ll = le.fit_transform(labels)
  target = np_utils.to_categorical(ll)
  return target

In [0]:
text = preprocess(df.copy())

In [0]:
filtered_text = Token_filter(text.copy())

In [0]:
filtered_text[0]

In [0]:
filtered_text = detokenize(filtered_text.copy())

In [0]:
max_len = 96
max_feat = 20000
tokenizer = Tokenizer(num_words=max_feat)
tokenizer.fit_on_texts(filtered_text)
sequence_train = tokenizer.texts_to_sequences(filtered_text)
data = pad_sequences(sequence_train,maxlen=max_len)
len_words = [len(words) for words in sequence_train]
print(max(len_words))
plt.hist(len_words, bins = np.arange(0,400,10))
plt.show()

In [0]:
target = prepare_labels(df)

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data, target , test_size = 0.05, random_state=0)

In [0]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(78524, 96)
(78524, 28)
(4133, 96)
(4133, 28)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0    27 10017   195   531     1 16136   135   517    43  2072   189
     8    15     5   128  1204   243    64   192   104  8583   346    11]


In [0]:
model = Sequential()
model.add(Embedding(20000, 96, input_length=96))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(28, activation='softmax'))


In [0]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = "adam",
             metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 96, 96)            1920000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 96, 96)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               78800     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 28)                2828      
Total params: 2,011,728
Trainable params: 2,011,728
Non-trainable params: 0
_________________________________________________________________


In [0]:
batch_size = 16
epochs = 5
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split = 0.10)

In [0]:
model.save('model.h5')

In [0]:
model.evaluate(x_test,y_test)

In [0]:
from sklearn.metrics import accuracy_score
print("Accuracy: " + str(round(accuracy_score(class_result,p)*100,4) )+"%")        
